In [1]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# 5-1. 코사인 유사도

In [5]:
doc1 = np.array([[0,1,1,1]])
doc2 = np.array([[1,0,1,1]])
doc3 = np.array([[2,0,2,2]])

In [14]:
cosine_similarity(doc1, doc2), cosine_similarity(doc1, doc3), cosine_similarity(doc2, doc3)

(array([[0.66666667]]), array([[0.66666667]]), array([[1.]]))

# 5-2. 유사도를 이용한 추천 시스템 구현
[영화 데이터](https://www.kaggle.com/rounakbanik/the-movies-dataset)

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
data = pd.read_csv('./data/movies_metadata.csv', low_memory=False)
data.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [4]:
data = data.head(20000)

In [5]:
data.overview.isna().sum() 

135

In [6]:
data.overview = data.overview.fillna('')

In [7]:
data.overview.isna().sum() 

0

In [8]:
v = TfidfVectorizer()
tfidf = v.fit_transform(data.overview.tolist()).toarray()

In [9]:
tfidf.shape, len(v.vocabulary_)

((20000, 47788), 47788)

In [10]:
sim = cosine_similarity(tfidf, tfidf)

In [12]:
sim.shape

(20000, 20000)

In [24]:
title_to_index = dict(zip(data.title, data.index))
index_title = dict(zip(data.index, data.title))

In [22]:
def get_recommendations(title, cosine_sim=sim):
    # 선택한 영화의 타이틀로부터 해당 영화의 인덱스를 받아온다.
    idx = title_to_index[title]

    # 해당 영화와 모든 영화와의 유사도를 가져온다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 영화들을 정렬한다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 영화를 받아온다.
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 영화의 인덱스를 얻는다.
    movie_indices = [idx[0] for idx in sim_scores]

    # 가장 유사한 10개의 영화의 제목을 리턴한다.
    return data['title'].iloc[movie_indices]

In [26]:
get_recommendations('Toy Story')

15348               Toy Story 3
2997                Toy Story 2
10301    The 40 Year Old Virgin
8327                  The Champ
1071      Rebel Without a Cause
11399    For Your Consideration
1932                  Condorman
3057            Man on the Moon
7254             Africa Screams
11606              Factory Girl
Name: title, dtype: object